## Data Transformation and Sentence Assembly

In this section, we load a dataset from a TSV file and process it to organize the data into sentences. Each token, along with its corresponding POS and NER tags. We then create a new DataFrame that includes the sentence text, tokens, and their associated tags. 


In [2]:
import pandas as pd

from pathlib import Path

# Use the current working directory as the base directory
base_dir = Path.cwd()

# Construct the file path
file_path = base_dir  / 'raw' / 'data.tsv'

# Read the TSV file
df = pd.read_csv(file_path, sep='\t', header=None, names=['token', 'pos_tag', 'ner_tag'])

# Initialize variables
sentence_id = 0
sentences = []
current_sentence = []

# Iterate through the dataframe rows
for index, row in df.iterrows():
    token = row['token']
    
    # Check if it's the end of a sentence
    if token.endswith('।'):
        current_sentence.append((sentence_id, token, row['pos_tag'], row['ner_tag']))
        sentences.extend(current_sentence)
        current_sentence = []
        sentence_id += 1
    else:
        current_sentence.append((sentence_id, token, row['pos_tag'], row['ner_tag']))

# Append remaining tokens
if current_sentence:
    sentences.extend(current_sentence)

# Create the final DataFrame
final_df = pd.DataFrame(sentences, columns=['sentence', 'token', 'pos_tag', 'ner_tag'])

# Generate sentence text for each sentence ID
sentence_texts = final_df.groupby('sentence')['token'].apply(lambda tokens: ' '.join(tokens)).to_dict()

# Add sentence text to DataFrame
final_df['sentence_text'] = final_df['sentence'].map(sentence_texts)

# Reorder columns
final_df = final_df[['sentence_text', 'token', 'pos_tag', 'ner_tag']]
final_df = final_df.dropna(subset=['pos_tag'])
# Display the DataFrame


In [3]:
final_df.head(20)

,sentence_text,token,pos_tag,ner_tag
1,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,শনিবার,NNP,B-D&T
2,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,(২৭,PUNCT,B-OTH
3,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,আগস্ট),NNP,B-D&T
4,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,রাতে,NNC,B-D&T
5,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,পটুয়াখালী,NNP,B-GPE
6,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,সদর,NNC,I-GPE
7,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,থানার,NNC,I-GPE
8,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,ভারপ্রাপ্ত,ADJ,B-PER
9,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,কর্মকর্তা,NNC,I-PER
10,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,(ওসি),PUNCT,B-OTH


In [4]:
final_df[final_df['pos_tag'] == 'PUNCT'].head(15)

,sentence_text,token,pos_tag,ner_tag
2,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,(২৭,PUNCT,B-OTH
10,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,(ওসি),PUNCT,B-OTH
93,"তিনি বলেন, “বাংলায় আয়কর আইন করতে আমরা স্টেকহোল...",“বাংলায়,PUNCT,B-OTH
111,শিক্ষা ও গবেষণার মান সমুন্নত রাখতে রাজশাহী বিশ...,(রাবি),PUNCT,B-OTH
124,২ নভেম্বর গ্রেগরীয় বর্ষপঞ্জী অনুসারে বছরের ৩০...,(অধিবর্ষে,PUNCT,B-OTH
264,"রিজভী আরও বলেন, ‘দেশে ৮০ শতাংশ মানুষের আয় বাড...",‘দেশে,PUNCT,B-OTH
305,সোমবার (২৩) জানুয়ারি দুপুরে জাতীয় প্রেস ক্লাবে...,(২৩),PUNCT,B-OTH
358,কিন্তু আলোকে কি ‘জিনিস’ বলা যায়? কিন্তু আলোকে ...,‘জিনিস’,PUNCT,B-OTH
389,"রাজু আলাউদ্দিন : [২] উচ্চ খেলাপির কারণ, আদায় প...",:,PUNCT,B-OTH
390,"রাজু আলাউদ্দিন : [২] উচ্চ খেলাপির কারণ, আদায় প...",[২],PUNCT,B-OTH


## Cleaning the PUNCT Tag Anomalies
Notice that the `PUNCT` tag includes not only punctuation marks but also words, numbers, and other elements that should not be classified as punctuation. To address this, we need to clean the data by retaining only actual punctuation marks and removing any other thing incorrectly tagged as `PUNCT`.

### Handling Tokens with Trailing Punctuation

Notice that if a token ends with a punctuation mark, it is not always tagged as `PUNCT`. For example, `আগস্ট)` is incorrectly tagged as `NNP` instead of `PUNCT`. To correct this, we need to remove any token where the last character is a punctuation mark but the token itself is not classified as `PUNCT`.


In [5]:
punctuation_chars = "।.,!?;:'\"()[]{}-—"

# Define a function to clean the token if pos_tag is PUNCT
def clean_punctuation_token(row):
    if row['pos_tag'] == 'PUNCT':
        # Keep only punctuation characters in the token
        row['token'] = ''.join([char for char in row['token'] if char in punctuation_chars])
    return row

# Apply the function to clean the tokens for PUNCT pos_tag
final_df = final_df.apply(clean_punctuation_token, axis=1)

# Define a function to remove trailing punctuation from a token
def remove_trailing_punctuation(token):
    if token and len(token) > 1 and token[-1] in punctuation_chars:
        return token[:-1]
    return token

# Apply the function to remove trailing punctuation from all tokens
final_df['token'] = final_df['token'].apply(remove_trailing_punctuation)

# Display the updated DataFrame
final_df.head()

,sentence_text,token,pos_tag,ner_tag
1,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,শনিবার,NNP,B-D&T
2,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,(,PUNCT,B-OTH
3,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,আগস্ট,NNP,B-D&T
4,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,রাতে,NNC,B-D&T
5,শনিবার (২৭ আগস্ট) রাতে পটুয়াখালী সদর থানার ভা...,পটুয়াখালী,NNP,B-GPE


### Grouping Tokens, POS Tags, and NER Tags by Sentence After Cleaning

After cleaning, stack the tokens, POS tags, and NER tags together, grouping them by sentence.


In [6]:
grouped = final_df.groupby('sentence_text').agg({
    'token': lambda x: list(x),
    'pos_tag': lambda x: list(x),
    'ner_tag': lambda x: list(x)
}).reset_index()

In [7]:
grouped.head()

,sentence_text,token,pos_tag,ner_tag
0,'এ স্টেট ইজ বর্ন' ছবিটির শুরু ১৯৪৭ সাল।,"[', স্টেট, ইজ, বর্ন, ছবিটির, শুরু, ১৯৪৭, সাল]","[PUNCT, NNP, NNP, NNP, NNC, NNC, QF, NNC]","[B-OTH, B-T&T, B-T&T, B-T&T, B-OTH, B-OTH, B-D..."
1,'কয়েদীরা দিনপঞ্জীর মতো দেয়ালে নিজেদের স্মৃতি ল...,"[', দিনপঞ্জীর, মতো, দেয়ালে, নিজেদের, স্মৃতি, ল...","[PUNCT, NNC, PP, NNC, PRO, NNC, VNF, VF]","[B-OTH, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH, B-O..."
2,( ২২ আগস্ট-২২ সেপ্টেম্বর): বৈদেশিক যোগাযোগে ...,"[(, ২২, আগস্ট-২২, সেপ্টেম্বর), বৈদেশিক, যোগায...","[PUNCT, QF, NNP, NNP, ADJ, NNC, NNC, ADJ, VNF,...","[B-OTH, B-D&T, I-D&T, I-D&T, B-OTH, B-OTH, B-D..."
3,"(আহমাদ, ইরওয়া হা/১১৬, ১১৭; মিশকাত হা/৩৯০) (আহম...","[(, ইরওয়া, হা/১১৬, ১১৭, মিশকাত, হা/৩৯০]","[PUNCT, NNP, NNC, QF, NNP, NNC]","[B-OTH, B-T&T, I-T&T, B-NUM, B-T&T, I-T&T]"
4,(উদ্ধৃতির ভেতরের সমস্ত বানান অপরিবর্তিত রাখা হ...,"[(, ভেতরের, সমস্ত, বানান, অপরিবর্তিত, রাখা, হয...","[PUNCT, NNC, ADJ, NNC, ADJ, NNC, VF]","[B-OTH, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH]"


### Save our dataset in JSON Format

In [8]:
import json
grouped = final_df.groupby('sentence_text').agg({
    'token': lambda x: list(x),
    'pos_tag': lambda x: list(x),
    'ner_tag': lambda x: list(x)
}).reset_index()

# Write each sentence as a separate JSON object, each on a new line
with open(f'{base_dir}/processed/processed_data.json', 'w', encoding='utf-8') as f:
    for _, row in grouped.iterrows():
        json_obj = {
            "tokens": row['token'],
            "pos_tag": row['pos_tag'],
            "ner_tags": row['ner_tag']
        }
        f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')